In [1]:
%load_ext autoreload
%autoreload 2

import plotly.io as pio
pio.renderers.default = 'iframe'

In [2]:
import pandas as pd
import plotly.express as px
import numpy as np
import pandas as pd
import re
from datetime import timedelta
import plotly.express as px

from op_analytics.datasources.defillama.dataaccess import DefiLlama
from op_analytics.coreutils.request import get_data, new_session

import urllib3
import warnings
pd.set_option('display.float_format', lambda x: '%.3f' % x)
urllib3.disable_warnings()
warnings.filterwarnings("ignore")

In [3]:
PATTERNS_TO_FILTER = [
    "-borrowed",
    "-vesting",
    "-staking",
    "-pool2",
    "-treasury",
    "-cex",
    "^treasury$",
    "^borrowed$",
    "^staking$",
    "^pool2$",
    "^pool2$",
    "polygon-bridge-&-staking",  # Added this as a full match
    ".*-cex$",  # Added this to match anything ending with -cex
]

CATEGORIES_TO_FILTER = ["CEX", "Chain"]

alignment_dict = {
    "Metis": "OP Stack fork",
    "Blast": "OP Stack fork",
    "Mantle": "OP Stack fork",
    "Zircuit": "OP Stack fork",
    "RSS3": "OP Stack fork",
    "Rollux": "OP Stack fork",
    "Ancient8": "OP Stack fork",
    "Manta": "OP Stack fork",
    "Cyber": "OP Chain",
    "Mint": "OP Chain",
    "Ham": "OP Chain",
    "Polynomial": "OP Chain",
    "Lisk": "OP Chain",
    "BOB": "OP Chain",
    "Mode": "OP Chain",
    "World Chain": "OP Chain",
    "Base": "OP Chain",
    "Kroma": "OP Chain",
    "Boba": "OP Chain",
    "Fraxtal": "OP Chain",
    "Optimism": "OP Chain",
    "Shape": "OP Chain",
    "Zora": "OP Chain"
}

alignment_df = pd.DataFrame(list(alignment_dict.items()), columns=["chain", "alignment"])

token_data = [
    {"token": "ETH", "token_category": "Native Asset"},
    {"token": "WETH", "token_category": "Native Asset"},
    {"token": "SOL", "token_category": "Native Asset"},
    {"token": "wBTC", "token_category": "Wrapped Assets"},
    {"token": "cbBTC", "token_category": "Wrapped Assets"},
    {"token": "MBTC", "token_category": "Wrapped Assets"},

    {"token": "stETH", "token_category": "Liquid Staking"},
    {"token": "wstETH", "token_category": "Liquid Staking"},
    {"token": "eETH", "token_category": "Liquid Restaking"},
    {"token": "weETH", "token_category": "Liquid Restaking"},
    {"token": "sfrxETH", "token_category": "Liquid Staking"},
    {"token": "rETH", "token_category": "Liquid Staking"},
    {"token": "mETH", "token_category": "Liquid Staking"},
    {"token": "rsETH", "token_category": "Liquid Restaking"},
    {"token": "cbETH", "token_category": "Liquid Staking"},
    {"token": "ezETH", "token_category": "Liquid Restaking"},
    {"token": "rswETH", "token_category": "Liquid Restaking"},
    {"token": "swETH", "token_category": "Liquid Staking"},
    {"token": "frxETH", "token_category": "Liquid Staking"},
    {"token": "ETHX", "token_category": "Liquid Staking"},
    {"token": "lsETH", "token_category": "Liquid Staking"},
    {"token": "oETH", "token_category": "Liquid Staking"},
    {"token": "EBTC", "token_category": "Liquid Restaking"},
    {"token": "LBTC", "token_category": "Liquid Restaking"},
    {"token": "SUPEROETHB", "token_category": "Liquid Staking"},
    {"token": "WSUPEROETHB", "token_category": "Liquid Staking"},
    {"token": "TETH", "token_category": "Liquid Staking"},
    {"token": "OSETH", "token_category": "Liquid Staking"},
    {"token": "cmETH", "token_category": "Liquid Restaking"},
    {"token": "WRSETH", "token_category": "Liquid Restaking"},
    {"token": "WEETH.BASE", "token_category": "Liquid Restaking"},
    
    {"token": "USDC", "token_category": "Stablecoins"},
    {"token": "USDT", "token_category": "Stablecoins"},
    {"token": "FDUSD", "token_category": "Stablecoins"},
    {"token": "PYUSD", "token_category": "Stablecoins"},
    {"token": "TUSD", "token_category": "Stablecoins"},
    {"token": "DAI", "token_category": "Stablecoins"},
    {"token": "USDE", "token_category": "Stablecoins"},
    {"token": "USDD", "token_category": "Stablecoins"},
    {"token": "FRAX", "token_category": "Stablecoins"},
    {"token": "EURC", "token_category": "Stablecoins"},
    {"token": "AGEUR", "token_category": "Stablecoins"},
    {"token": "USDS", "token_category": "Stablecoins"},
    {"token": "USDB", "token_category": "Stablecoins"},
    {"token": "DOLA", "token_category": "Stablecoins"},
    {"token": "SUSDE", "token_category": "Stablecoins"},
    {"token": "USD0++", "token_category": "Stablecoins"},
    {"token": "USD0", "token_category": "Stablecoins"},
    {"token": "SUSD", "token_category": "Stablecoins"},
    {"token": "CRVUSD", "token_category": "Stablecoins"},
    {"token": "USDC+", "token_category": "Stablecoins"},
    {"token": "USDZ", "token_category": "Stablecoins"},
    {"token": "STAR", "token_category": "Stablecoins"},
    {"token": "USDBC", "token_category": "Stablecoins"},
    {"token": "USD+", "token_category": "Stablecoins"},
    {"token": "CDXUSD", "token_category": "Stablecoins"},
    {"token": "HYUSD", "token_category": "Stablecoins"},
    {"token": "STAR", "token_category": "Stablecoins"},
    {"token": "EURS", "token_category": "Stablecoins"},
    {"token": "AXLEUROC", "token_category": "Stablecoins"},


    # Solana Liquid staking
    {"token": "MSOL", "token_category": "Liquid Staking"},
    {"token": "JUPSOL", "token_category": "Liquid Staking"},
    {"token": "BNSOL", "token_category": "Liquid Staking"},
    {"token": "SSOL", "token_category": "Liquid Restaking"},
    {"token": "BBSOL", "token_category": "Liquid Restaking"},
    {"token": "LAINESOL", "token_category": "Liquid Staking"},
    {"token": "STSOL", "token_category": "Liquid Staking"},
    {"token": "STRONGSOL", "token_category": "Liquid Staking"},
    {"token": "HUBSOL", "token_category": "Liquid Staking"},
    {"token": "PATHSOL", "token_category": "Liquid Staking"},
    {"token": "STEPSOL", "token_category": "Liquid Staking"},
    {"token": "EDGESOL", "token_category": "Liquid Staking"},
    {"token": "JITOSOL", "token_category": "Liquid Staking"},
    {"token": "DSOL", "token_category": "Liquid Staking"},
    {"token": "BONKSOL", "token_category": "Liquid Staking"},
    {"token": "VSOL", "token_category": "Liquid Staking"},
    {"token": "HSOL", "token_category": "Liquid Staking"},
    # {"token": "ARB", "token_category": "Layer 2 Token"},
    # {"token": "OP", "token_category": "Layer 2 Token"},
    # {"token": "MODE", "token_category": "Layer 2 Token"},
]

token_categories = pd.DataFrame(token_data)

token_categories["token"] = token_categories["token"].str.upper()


mapping = {
    "Dexes": "Trading",
    "Liquidity manager": "Yield",
    "Derivatives": "Derivatives",
    "Yield Aggregator": "Yield",
    "Indexes": "Yield",
    "Bridge": "Trading",
    "Leveraged Farming": "Yield",
    "Cross Chain": "Trading",
    "CDP": "Lending",
    "Farm": "Yield",
    "Options": "Trading",
    "DCA Tools": "Trading",
    "Services": "TradFi/Fintech",
    "Chain": "TradFi/Fintech",
    "Privacy": "TradFi/Fintech",
    "RWA": "TradFi/Fintech",
    "Payments": "TradFi/Fintech",
    "Launchpad": "TradFi/Fintech",
    "Synthetics": "Derivatives",
    "SoFi": "TradFi/Fintech",
    "Prediction Market": "Trading",
    "Token Locker": "Yield",
    "Yield Lottery": "Yield",
    "Algo-Stables": "Stablecoins",
    "DEX Aggregator": "Trading",
    "Liquid Restaking": "Restaking/Liquid Restaking",
    "Governance Incentives": "Yield",
    "Restaking": "Restaking/Liquid Restaking",
    "Liquid Staking": "Liquid Staking",
    "Uncollateralized Lending": "Lending",
    "Managed Token Pools": "Trading",
    "Insurance": "TradFi/Fintech",
    "NFT Marketplace": "Trading",
    "NFT Lending": "Lending",
    "Options Vault": "Trading",
    "NftFi": "Trading",
    "Basis Trading": "Trading",
    "Bug Bounty": "TradFi/Fintech",
    "OTC Marketplace": "Trading",
    "Reserve Currency": "Stablecoins",
    "Gaming": "Other",
    "AI Agents": "TradFi/Fintech",
    "Treasury Manager": "TradFi/Fintech",
    "CDP Manager": "Lending",
    "Decentralized Stablecoin": "Stablecoins",
    "Restaked BTC": "Restaking/Liquid Restaking",
    "RWA Lending": "Lending",
    "Staking Pool": "Staking/Liquid Staking",
    "CeDeFi": "TradFi/Fintech",
    "Staking": "Staking/Liquid Staking",
    "Oracle": "Other",
    "Ponzi": "Other",
    "Anchor BTC": "Other",
    "Decentralized BTC": "Other",
    "CEX": "Other",
    "Lending": "Lending"
}


In [4]:
from op_analytics.coreutils.duckdb_inmem.client import init_client
from op_analytics.coreutils.duckdb_inmem.localcopy import dump_local_copy, load_local_copy
from op_analytics.datasources.defillama.dataaccess import DefiLlama

duckdb_client = init_client()

2024-12-17 09:51:36 [info     ] loaded vault from .env file    filename=vault.py lineno=32
2024-12-17 09:51:36 [debug    ] loaded vault: 17 items         filename=vault.py lineno=76


- Pull this data fresh, should be okay to leave protocol metadata date as-is
- I would use "2024-11-30" as your latest date, we ran into a few data issues with more recent data
- Make sure your secrets are up to date, Pedro updated them on Dec 2nd to work with GCS
- There could be lingering data issues but Pedro addressed a bunch today

In [5]:
view1 = DefiLlama.PROTOCOLS_TOKEN_TVL.read(min_date="2023-12-01")

df_protocol_tvl = duckdb_client.sql(
f"""
SELECT
    dt,
    protocol_slug,
    chain,
    token,
    app_token_tvl,
    app_token_tvl_usd
FROM {view1}
""").to_df()

2024-12-17 09:51:37 [info     ] querying markers for 'defillama/protocols_token_tvl_v1' DateFilter(min_date=datetime.date(2023, 12, 1), max_date=None, datevals=None) filename=dailydata.py lineno=107
2024-12-17 09:51:37 [debug    ] connecting to OPLABS Clickhouse client... filename=client.py lineno=25
2024-12-17 09:51:38 [debug    ] initialized OPLABS Clickhouse client. filename=client.py lineno=37
2024-12-17 09:51:38 [info     ] 480 markers found              filename=dailydata.py lineno=121
2024-12-17 09:51:38 [info     ] 383 distinct paths             filename=dailydata.py lineno=127
2024-12-17 09:51:39 [info     ] registered view: 'defillama_protocols_token_tvl_v1' using 383 parquet paths filename=client.py lineno=53
┌──────────────────────────────────┐
│               name               │
│             varchar              │
├──────────────────────────────────┤
│ defillama_protocols_token_tvl_v1 │
└──────────────────────────────────┘



In [6]:
view2 = DefiLlama.PROTOCOLS_METADATA.read(min_date="2024-12-15")

df_metadata = duckdb_client.sql(
f"""
SELECT 
    protocol_name,
    protocol_slug,
    protocol_category,
    parent_protocol,
    CASE WHEN misrepresented_tokens = 'True' THEN 1
        WHEN misrepresented_tokens = 'False' THEN 0
        ELSE 0
    END AS misrepresented_tokens
FROM {view2}
""").to_df()

2024-12-17 09:52:11 [info     ] querying markers for 'defillama/protocols_metadata_v1' DateFilter(min_date=datetime.date(2024, 12, 15), max_date=None, datevals=None) filename=dailydata.py lineno=107
2024-12-17 09:52:12 [info     ] 3 markers found                filename=dailydata.py lineno=121
2024-12-17 09:52:12 [info     ] 3 distinct paths               filename=dailydata.py lineno=127
2024-12-17 09:52:13 [info     ] registered view: 'defillama_protocols_metadata_v1' using 3 parquet paths filename=client.py lineno=53
┌──────────────────────────────────┐
│               name               │
│             varchar              │
├──────────────────────────────────┤
│ defillama_protocols_metadata_v1  │
│ defillama_protocols_token_tvl_v1 │
└──────────────────────────────────┘



In [7]:
# this didn't yield useful results (pun intended)

YIELD_ENDPOINT = "https://yields.llama.fi/pools"
session = new_session()
yield_data = get_data(session, YIELD_ENDPOINT)

yield_lists = []

for yield_pool in yield_data["data"]:
    chain = yield_pool["chain"]
    project = yield_pool["project"]
    pool = yield_pool["pool"]
    symbol = yield_pool["symbol"]
    underlying_tokens = yield_pool["underlyingTokens"] 
    yield_lists.append(
        {
            "chain_name": chain,
            "protocol_slug": project,
            "pool": pool,
            "symbol": symbol,
            "underlying_tokens": underlying_tokens
        }
    )

df_yield = pd.DataFrame(yield_lists)


2024-12-17 09:52:15 [info     ] Fetched from https://yields.llama.fi/pools: 0.69 seconds filename=request.py lineno=81


In [8]:
ethena_pool = df_yield[df_yield.protocol_slug == "ethena-usde"].pool.to_list()
aave_pools = df_yield[
    (df_yield.protocol_slug == "aave-v3")
    & (df_yield.chain_name == "Ethereum")
].pool.to_list()

yield_pools = ethena_pool + aave_pools


In [9]:
YIELD_POOL_ENDPOINT = "https://yields.llama.fi/chart/{pool}"
session = new_session()
yield_data = get_data(session, YIELD_ENDPOINT)

pool_dfs = []
for pool in yield_pools:
    url = YIELD_POOL_ENDPOINT.format(pool=pool)
    pool_chart = get_data(session, url)
    df_pool = pd.DataFrame(pool_chart["data"])
    df_pool["pool"] = pool
    
    pool_dfs.append(df_pool)



2024-12-17 09:52:16 [info     ] Fetched from https://yields.llama.fi/pools: 0.55 seconds filename=request.py lineno=81
2024-12-17 09:52:16 [info     ] Fetched from https://yields.llama.fi/chart/66985a81-9c51-46ca-9977-42b4fe7bc6df: 0.05 seconds filename=request.py lineno=81
2024-12-17 09:52:16 [info     ] Fetched from https://yields.llama.fi/chart/db678df9-3281-4bc2-a8bb-01160ffd6d48: 0.04 seconds filename=request.py lineno=81
2024-12-17 09:52:16 [info     ] Fetched from https://yields.llama.fi/chart/e6435aae-cbe9-4d26-ab2c-a4d533db9972: 0.04 seconds filename=request.py lineno=81
2024-12-17 09:52:16 [info     ] Fetched from https://yields.llama.fi/chart/7e382157-b1bc-406d-b17b-facba43b716e: 0.06 seconds filename=request.py lineno=81
2024-12-17 09:52:16 [info     ] Fetched from https://yields.llama.fi/chart/d3be212c-f337-40e2-87fc-fb636a3de38f: 0.04 seconds filename=request.py lineno=81
2024-12-17 09:52:16 [info     ] Fetched from https://yields.llama.fi/chart/29932dea-cd71-44c3-95bd-3e

In [10]:
df_pools = pd.concat(pool_dfs)

In [12]:
df_pools = pd.merge(
    df_pools,
    df_yield,
    on="pool",
    how="left"
)

In [13]:
pool_counts = df_pools.groupby(["pool", "symbol"]).agg({"timestamp": "nunique"}).reset_index().sort_values(by="symbol")

In [15]:
symbols = [
    'CBBTC',
    'CBETH',
    'CRVUSD',
    'ETHX',
    'FRAX',
    'LUSD',
    'OSETH',
    'PYUSD',
    'RETH',
    'SUSDE',
    'TBTC',
    'USDC',
    'USDE',
    'USDS',
    'USDT',
    'WBTC',
    'WEETH',
    'WETH',
    'WSTETH'
]

In [16]:
pools_to_use = (
    pool_counts[
    (pool_counts.timestamp > 40) # manual filter, need to understand what's happening 
    & pool_counts.symbol.isin(symbols)
    & (pool_counts.pool != "29932dea-cd71-44c3-95bd-3e1525f4e3dd")
    ].pool.to_list()
)

In [19]:
df_pools["slug_symbol"] = df_pools["symbol"] + "-" + df_pools["protocol_slug"]

In [22]:
df_pools = df_pools.sort_values(by=["pool", "timestamp"])
df_pools["timestamp"] = pd.to_datetime(df_pools["timestamp"], utc=True)

df_pools["apy_7d_rolling_avg"] = df_pools.groupby("pool").apply(
    lambda group: group.sort_values("timestamp").rolling("7d", on="timestamp")["apy"].mean()
).reset_index(level=0, drop=True)

In [23]:
df_pools["timestamp"] = pd.to_datetime(df_pools["timestamp"])

# Create the line plot
fig = px.line(
    df_pools[
       df_pools.pool.isin(pools_to_use)
        & (df_pools.timestamp >= "2024-09-01")
        & (df_pools.symbol.isin(["USDE", "SUSDE", "USDT", "USDC"]))
    ]
    ,
    x="timestamp",
    y="apy_7d_rolling_avg",
    color="symbol",
    title="Yield pool APY over time",
    labels={"dt": "Date", "app_token_tvl_usd": "TVL (USD)", "symbol": "Token"},
)

# Customize layout for better visualization
fig.update_layout(
    template="plotly_white",
    xaxis_title="Date",
    yaxis_title="TVL (USD)",
    legend_title="Token",
    margin=dict(t=50, l=25, r=25, b=50),
)

# Show plot
fig.show()

In [25]:
# drop duplicates due to an ongoing data upload issue
df_all = pd.merge(
    df_metadata.drop_duplicates(), 
    df_protocol_tvl.drop_duplicates(), 
    on="protocol_slug",
    how="left"
)


In [26]:
# Merge data and join alignment and token categories
df_all = pd.merge(df_all, alignment_df, on="chain", how="left")
df_all["alignment"] = df_all["alignment"].fillna("Other")
df_all = pd.merge(df_all, token_categories, on="token", how="left")
df_all["token_category"] = df_all["token_category"].fillna("Other")


In [27]:
# Chain level misrepresented tokens
df_misrep = (
    df_all[df_all.dt == df_all["dt"].max()-pd.Timedelta(days=1)]
    [["protocol_slug", "chain", "misrepresented_tokens", "token"]]
    .groupby(["protocol_slug", "chain", "misrepresented_tokens"])
    .agg(
        token_count=("token", "nunique"),
        has_usdt=("token", lambda x: 1 if "USDT" in x.values else 0)
    )
    .reset_index()
)

df_misrep["chain_misrepresented_tokens"] = (
    (df_misrep["misrepresented_tokens"] == 1) 
    & (df_misrep["token_count"] == 1) 
    & (df_misrep["has_usdt"] == 1)
).astype(int)

df_all = pd.merge(
    df_all, 
    df_misrep[["protocol_slug", "chain", "chain_misrepresented_tokens"]], 
    on=["protocol_slug", "chain"],
    how="left"
)

In [28]:
protocol_slug = "aave-v3"

In [29]:
df_borrow = df_all[
    (df_all.dt >= "2023-12-15")
    # & (df_all.chain.str.contains("borrow"))
    & (df_all.chain == "Ethereum-borrowed")
    & (df_all.protocol_slug == protocol_slug)
    # & (df_all.token_category == "Stablecoins")
    # & (df_all.token.isin(["SUSDE", "USDT", "USDC", "DAI"]))
].groupby(["dt", "protocol_name", "token"]).agg({"app_token_tvl_usd": "sum"}).reset_index()

In [30]:
df_borrow["token"] = df_borrow["token"] + "_borrowed"

In [31]:
df_lending = df_all[
    (df_all.dt >= "2023-12-15")
    # & (df_all.chain.str.contains("borrow"))
    & (df_all.chain == "Ethereum")
    & (df_all.protocol_slug == protocol_slug)
    # & (df_all.token_category == "Stablecoins")
    & (df_all.token.isin(["SUSDE", "USDT", "USDC"]))

].groupby(["dt", "protocol_name", "token"]).agg({"app_token_tvl_usd": "sum"}).reset_index()

In [32]:
df_lending_borrow = pd.concat([df_lending, df_borrow])

In [33]:
df_lending_borrow["dt"] = pd.to_datetime(df_lending_borrow["dt"])

# Create the line plot
fig = px.line(
    df_lending_borrow[
        (df_lending_borrow.token.isin(["SUSDE", "USDT", "USDC", "USDE"]))
        & (df_lending_borrow.dt >= "2024-06-15")
    ]
    ,
    x="dt",
    y="app_token_tvl_usd",
    color="token",
    title="App Token TVL (USD) Over Time",
    labels={"dt": "Date", "app_token_tvl_usd": "TVL (USD)", "token": "Token"},
)

# Customize layout for better visualization
fig.update_layout(
    template="plotly_white",
    xaxis_title="Date",
    yaxis_title="TVL (USD)",
    legend_title="Token",
    margin=dict(t=50, l=25, r=25, b=50),
)

# Show plot
fig.show()

In [42]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

# Filter data for the pools and lending data
df_pools_filtered = df_pools[
    df_pools["pool"].isin(pools_to_use) &
    (df_pools["timestamp"] >= "2024-06-01") &
    (df_pools["symbol"].isin(["USDE", "SUSDE", "USDT", "USDC"]))
]

df_lending_borrow_filtered = df_lending_borrow[
    (df_lending_borrow["token"] == "SUSDE") &
    (df_lending_borrow["dt"] >= "2024-06-15")
]

# Create the figure for the SUSDE Lending TVL first (dashed black line)
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=df_lending_borrow_filtered["dt"],
        y=df_lending_borrow_filtered["app_token_tvl_usd"],
        mode="lines",
        name="sUSDe AAVE Lending TVL",
        line=dict(color="black", width=2, dash="dash"),
        yaxis="y2"
    )
)

# Add the lines for the lending APY
color_mapping = {
    "SUSDE": "black",         # SUSDE Ethena APY (solid black)
    "USDE": "gray",           # USDE Lending APY (gray)
    "USDT": "green",          # USDT Lending APY (green)
    "USDC": "blue"            # USDC Lending APY (blue)
}

label_mapping = {
    "SUSDE": "sUSDe Ethena Native APY",
    "USDE": "USDe AAVE Lending APY",
    "USDT": "USDT AAVE Lending APY",
    "USDC": "USDC AAVE Lending APY"
}

for symbol in ["SUSDE", "USDE", "USDT", "USDC"]:
    filtered_df = df_pools_filtered[df_pools_filtered["symbol"] == symbol]
    fig.add_trace(
        go.Scatter(
            x=filtered_df["timestamp"],
            y=filtered_df["apy_7d_rolling_avg"],
            mode="lines",
            name=label_mapping[symbol],
            line=dict(color=color_mapping[symbol], width=2)
        )
    )

# Update layout to include a secondary y-axis and adjust the legend position
fig.update_layout(
    template="plotly_white",
    xaxis_title="Date",
    yaxis_title="APY (%)",
    yaxis2=dict(
        title="Lending TVL (USD)",
        overlaying="y",
        side="right",
        showgrid=False
    ),
    legend=dict(
        x=1.05,  # Move the legend slightly to the right
        y=1,     # Align the legend to the top
        xanchor="left", 
        yanchor="top"
    ),
    margin=dict(t=50, l=25, r=50, b=50),
)

# Show plot
fig.show()
